# Consolidated Performance Table 

## 1. Imports, Config and Loads

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

sns.set_style("whitegrid")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 10

# config
METRICS_FILE = Path("../bench/metrics.csv")
OUTPUT_DIR = Path("../bench")
FIGURES_DIR = OUTPUT_DIR / "figures"
FIGURES_DIR.mkdir(exist_ok=True)

METRICS = ['handshake_ms', 'latency_ms', 'throughput_mbps', 'cpu_pct', 'mem_mb', 'packet_loss_pct']

BANDWIDTH_LIMITS = {
    'local': 1000,      # 1 Gbps 
    'realistic': 10,    # 10 Mbps
    'adverse': 5,       # 5 Mbps
    'highlat': 2        # 2 Mbps
}

print(f"  Data file: {METRICS_FILE}")
print(f"  Output: {OUTPUT_DIR}")
print(f"  Figures: {FIGURES_DIR}")

  Data file: ..\bench\metrics.csv
  Output: ..\bench
  Figures: ..\bench\figures


In [ ]:
df = pd.read_csv(METRICS_FILE)

numeric_cols = ['latency_ms', 'handshake_ms', 'throughput_mbps', 'cpu_pct', 'mem_mb', 'packet_loss_pct']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

print(f"  Columns: {list(df.columns)}")

print("\nData Distribution:")
display(df.groupby(['protocol', 'scheme', 'cond_profile']).size().unstack(fill_value=0))


  Columns: ['timestamp', 'protocol', 'scheme', 'kem', 'signature', 'client_ip', 'server_ip', 'cond_profile', 'latency_ms', 'handshake_ms', 'throughput_mbps', 'cpu_pct', 'mem_mb', 'packet_loss_pct']

Data Distribution:


cond_profile       adverse  highlat  local  realistic
protocol  scheme                                     
IPsec     classic      750      750    750        750
          hybrid       750      750    750        750
OpenVPN   classic      750      750    750        750
          hybrid       750      750    750        750
WireGuard classic      750      750    750        750
          hybrid       750      750    750        750

## 2. Outliers Filter

In [ ]:
def remove_outliers(series, multiplier=10):
    """
    Remove extreme outliers using IQR method.

    Parameters:
    - series: pandas Series with numeric data
    - multiplier: IQR multiplier (default 10 = only truly absurd values)
                Use 1.5 for standard outliers, 3 for extreme, 10+ for absurdly extreme

    Returns:
    - Series without extreme outliers
    """
    if len(series) == 0:
        return series

    Q1 = series.quantile(0.25)
    Q3 = series.quantile(0.75)
    IQR = Q3 - Q1

    lower_bound = Q1 - multiplier * IQR
    upper_bound = Q3 + multiplier * IQR

    filtered = series[(series >= lower_bound) & (series <= upper_bound)]

    return filtered

 Outlier filtering function defined
  Method: IQR with multiplier=10 (absurdly extreme outliers only)
  Applied only to: handshake_ms, latency_ms
  NOT applied to: throughput_mbps, cpu_pct, mem_mb, packet_loss_pct



## 3: Main Consolidated Table (Mean ± SD)


### 3.1 Outlier Filtering

In [ ]:
groupby_cols = ['cond_profile', 'protocol', 'scheme']

# metrics with necessary outlier
FILTER_METRICS = ['handshake_ms', 'latency_ms']

removed_outliers_list = []

stats_list = []
for metric in METRICS:
    if metric in FILTER_METRICS:
        for (profile, protocol, scheme), group in df.groupby(groupby_cols):
            values = group[metric].dropna()
            if len(values) > 0:
                Q1 = values.quantile(0.25)
                Q3 = values.quantile(0.75)
                IQR = Q3 - Q1
                lower_bound = Q1 - 10 * IQR
                upper_bound = Q3 + 10 * IQR
                
                outliers = group[~((group[metric] >= lower_bound) & (group[metric] <= upper_bound))]
                if len(outliers) > 0:
                    for idx, row in outliers.iterrows():
                        removed_outliers_list.append({
                            'cond_profile': profile,
                            'protocol': protocol,
                            'scheme': scheme,
                            'metric': metric,
                            'value': row[metric],
                            'Q1': Q1,
                            'Q3': Q3,
                            'IQR': IQR,
                            'lower_bound': lower_bound,
                            'upper_bound': upper_bound,
                            'group_mean': values.mean(),
                            'group_median': values.median(),
                            'timestamp': row['timestamp']
                        })
        
        metric_stats = df.groupby(groupby_cols)[metric].agg([
            'count',
            'mean',
            'std',
            'median',
            ('q25', lambda x: x.quantile(0.25)),
            ('q75', lambda x: x.quantile(0.75)),
            ('p95', lambda x: x.quantile(0.95)),
            'min',
            'max',
            ('mean_filtered', lambda x: remove_outliers(x.dropna()).mean()),
            ('count_filtered', lambda x: len(remove_outliers(x.dropna()))),
            ('outliers_removed', lambda x: len(x.dropna()) - len(remove_outliers(x.dropna())))
        ]).reset_index()
    else: 
        metric_stats = df.groupby(groupby_cols)[metric].agg([
            'count',
            'mean',
            'std',
            'median',
            ('q25', lambda x: x.quantile(0.25)),
            ('q75', lambda x: x.quantile(0.75)),
            ('p95', lambda x: x.quantile(0.95)),
            'min',
            'max',
            ('mean_filtered', lambda x: x.dropna().mean()),  
            ('count_filtered', lambda x: len(x.dropna())),    
            ('outliers_removed', lambda x: 0)                 
        ]).reset_index()

    # cv: coefficient of variation (filtered mean)
    metric_stats['cv'] = (metric_stats['std'] / metric_stats['mean_filtered'].abs()) * 100

    metric_stats.columns = ['cond_profile', 'protocol', 'scheme'] + [f'{metric}_{col}' for col in metric_stats.columns[3:]]
    stats_list.append(metric_stats.set_index(groupby_cols))

stats_df = pd.concat(stats_list, axis=1).reset_index()
stats_df['n_samples'] = df.groupby(groupby_cols).size().values

# sort by profile
profile_order = {'local': 0, 'realistic': 1, 'adverse': 2, 'highlat': 3}
stats_df['profile_order'] = stats_df['cond_profile'].map(profile_order)
stats_df = stats_df.sort_values(['profile_order', 'protocol', 'scheme']).drop(columns=['profile_order'])

if removed_outliers_list:
    outliers_df = pd.DataFrame(removed_outliers_list)
    outliers_df['ratio_to_mean'] = outliers_df['value'] / outliers_df['group_mean']
    outliers_df = outliers_df.sort_values('ratio_to_mean', ascending=False)
    
    numeric_cols = ['value', 'Q1', 'Q3', 'IQR', 'lower_bound', 'upper_bound', 'group_mean', 'group_median', 'ratio_to_mean']
    for col in numeric_cols:
        outliers_df[col] = outliers_df[col].round(2)
    
    outliers_file = OUTPUT_DIR / "removed_outliers.csv"
    outliers_df.to_csv(outliers_file, index=False)
    print(f"Total outliers removed: {len(outliers_df)}")
    print("\n5 most extreme outliers (ratio to group mean):")
    display(outliers_df[['cond_profile', 'protocol', 'scheme', 'metric', 'value', 'group_mean', 'group_median', 'ratio_to_mean']].head())
else:
    print(" No outliers were removed")

print("\nOutliers removed (only handshake_ms and latency_ms):")
for metric in FILTER_METRICS:
    if f'{metric}_outliers_removed' in stats_df.columns:
        total_outliers = stats_df[f'{metric}_outliers_removed'].sum()
        print(f"  {metric}: {int(total_outliers)} extreme values removed")

  Total outliers removed: 1165

Top 5 most extreme outliers (ratio to group mean):


,cond_profile,protocol,scheme,metric,value,group_mean,group_median,ratio_to_mean
791,realistic,IPsec,hybrid,handshake_ms,8867006.78,12560.19,332.37,705.96
731,local,IPsec,hybrid,handshake_ms,82814.64,445.51,233.09,185.89
105,adverse,IPsec,hybrid,handshake_ms,88514.28,827.68,527.97,106.94
734,local,IPsec,hybrid,handshake_ms,47311.53,445.51,233.09,106.20
625,highlat,IPsec,hybrid,handshake_ms,116646.17,1429.18,830.35,81.62



Outliers removed (only handshake_ms and latency_ms):
  handshake_ms: 676 valores extremos removidos
  latency_ms: 233 valores extremos removidos


,cond_profile,protocol,scheme,n_samples
12,local,IPsec,classic,750
13,local,IPsec,hybrid,750
14,local,OpenVPN,classic,750
15,local,OpenVPN,hybrid,750
16,local,WireGuard,classic,750
17,local,WireGuard,hybrid,750
18,realistic,IPsec,classic,750
19,realistic,IPsec,hybrid,750
20,realistic,OpenVPN,classic,750
21,realistic,OpenVPN,hybrid,750


### 3.2 Main Table Creation

In [ ]:
def format_mean_sd(row, metric):
    mean = row[f'{metric}_mean_filtered']  
    std = row[f'{metric}_std']
    n = row[f'{metric}_count_filtered']     

    if pd.isna(mean) or n == 0:
        return "NA"
    elif pd.isna(std):
        return f"{mean:.2f}"
    else:
        return f"{mean:.2f} ± {std:.2f}"

main_table = stats_df[['cond_profile', 'protocol', 'scheme', 'n_samples']].copy()

for metric in METRICS:
    main_table[metric] = stats_df.apply(lambda row: format_mean_sd(row, metric), axis=1)

display(main_table)

output_file = OUTPUT_DIR / "table_1_main_consolidated.csv"
main_table.to_csv(output_file, index=False)
print(f"\nSaved to: {output_file}")

Main consolidated table created


,cond_profile,protocol,scheme,n_samples,handshake_ms,latency_ms,throughput_mbps,cpu_pct,mem_mb,packet_loss_pct
12,local,IPsec,classic,750,311.93 ± 21.01,4.52 ± 0.56,145.56 ± 4.30,90.28 ± 4.85,6.75 ± 0.29,0.00 ± 0.00
13,local,IPsec,hybrid,750,235.35 ± 3580.96,4.40 ± 0.68,148.20 ± 6.48,89.64 ± 4.91,5.40 ± 0.17,0.80 ± 8.91
14,local,OpenVPN,classic,750,693.50 ± 44.44,5.34 ± 0.54,40.46 ± 1.55,77.69 ± 3.95,4.67 ± 0.23,0.00 ± 0.00
15,local,OpenVPN,hybrid,750,3624.95 ± 118.92,7.93 ± 0.90,24.84 ± 1.50,36.66 ± 2.23,4.08 ± 0.22,0.00 ± 0.00
16,local,WireGuard,classic,750,5.29 ± 2.77,5.29 ± 0.62,164.09 ± 20.85,16.48 ± 1.44,3.11 ± 0.10,0.00 ± 0.00
17,local,WireGuard,hybrid,750,5726.75 ± 184.83,5.31 ± 1.18,138.96 ± 16.79,11.54 ± 0.94,4.58 ± 0.40,0.00 ± 0.00
18,realistic,IPsec,classic,750,411.11 ± 26.27,101.86 ± 3.80,8.13 ± 0.61,30.37 ± 1.68,6.52 ± 0.05,0.00 ± 0.00
19,realistic,IPsec,hybrid,750,334.17 ± 324670.05,102.25 ± 3.89,7.73 ± 1.02,27.18 ± 2.13,5.18 ± 0.03,0.53 ± 7.29
20,realistic,OpenVPN,classic,750,1135.03 ± 50.28,102.76 ± 3.95,8.12 ± 0.60,30.35 ± 1.52,3.90 ± 0.06,0.00 ± 0.00
21,realistic,OpenVPN,hybrid,750,4772.20 ± 131.69,105.71 ± 3.72,2.83 ± 2.08,9.68 ± 4.67,3.26 ± 0.27,0.00 ± 0.00



Saved to: ..\bench\table_1_main_consolidated.csv
